In [1]:
import pandas as pd

# Centers CSV

In [6]:
# First, we need to clean the horribly formatted initial dataset into something usable.

path = 'resources/centers-uncleaned.csv'
with open(path, 'r') as csv_file:
    output = csv_file.read()
output = output.split('\n')

all_name = []
all_city = []
all_state = []
all_zip = []

for i in range(len(output)):
    info = output[i].split(';')
    try:
        name = info[1]
        # This loop searches for any entry that looks like a city / state / zip combination; if one is found it's saved, otherwise we ignore that row, since we don't have any useful information about that center.
        for j in range(2, len(info) + 1):
            try:
                if(all(x in '1234567890' for x in info[j][-5:])):
                    area = info[j]
                    break
            except:
                pass
        
        city = area.split(',')[0]
        state = area.split(',')[1][1:].split()[0]
        zipcode = area.split(',')[1][1:].split()[1]
        all_name.append(name)
        all_city.append(city)
        all_state.append(state)
        all_zip.append(zipcode)
    except:
        pass
    
# Create a nicely formatted dataframe to work with

centers_df = pd.DataFrame({'Name' : all_name, 'City' : all_city, 'State' : all_state, 'Zip' : all_zip})

# add lat / lon columns for the zip code so that 
zips_df = pd.read_csv('resources/zips.csv', dtype = {'ZIP' : 'str'})
zips_df = zips_df.rename(columns = {'ZIP' : 'Zip', 'LAT' : 'Lat', 'LNG' : 'Lon'})
centers_df = centers_df.merge(zips_df, on = 'Zip', how = 'inner')

out_path = 'resources/centers.csv'
centers_df.to_csv(out_path)

centers_df.head()

state_df = centers_df.loc[~centers_df['State'].isin(['GU', 'VI', 'AS'])].groupby(by = 'State')['Name'].count()
state_df.to_csv('resources/centers_bystate.csv')
state_df.head()

State
AK     80
AL    133
AR    204
AZ    305
CA    670
Name: Name, dtype: int64

# State Information CSV

In [3]:
# Import and read CSVs
centers_data = "Resources/centers_bystate.csv"
suicide_data = "Resources/suicide_states.csv"
suicide_df = pd.read_csv(suicide_data)
centers_df = pd.read_csv(centers_data, header=None, index_col=[0])

# Create headers and set index in "Centers" dataframe
centers_df.index.names = ['State']
centers_df = centers_df.rename(columns={1:"Centers"})

# Replace state name with state abbreviation in "Suicide" dataframe
suicide_df["State"] = suicide_df["State"].replace({
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
})
suicide_df_17 = suicide_df.loc[suicide_df["Year"]==2017]
suicide_data = suicide_df_17[["State", "Crude Rate", "Population"]]
centers_suicide = pd.merge(suicide_data, centers_df, on="State")
centers_suicide.to_csv("Resources/centers_vs_suicide.csv", index=False, header=True)
centers_suicide.head()

,State,Crude Rate,Population,Centers
0,AL,17.15,4874747,133
1,AK,27.03,739795,80
2,AZ,18.91,7016270,305
3,AR,21.00,3004279,204
4,CA,10.91,39536653,670


# MHS CSV

In [5]:
# read CSVs into dataframes
data_file = "resources/MHSS-2010-data.csv"
data_pd = pd.read_csv(data_file, sep='\t')

states_file = "resources/state_census_codes.csv"
states_id = pd.read_csv(states_file)

# create a reduced dataframe from Mental Health Services Survey file
mh_services = data_pd[["CASEID","STFIPS", "Ownership", "MHSuicide"]]

# create merged dataframe to add in states and regions
mhs = pd.merge(mh_services, states_id, on = "STFIPS")

# Convert response codes to values for facility ownership and suicide prevention program
mhs["MHSuicide"] = mhs["MHSuicide"].replace({0:"No",1:"Yes",(-1):"Unknown"})

mhs["Ownership"] = mhs["Ownership"].replace({1:"Private for-profit",
                                             2:"Private non-profit",
                                             3:"State or Regional Government", 
                                             5:"State or Regional Government",
                                             6:"State or Regional Government", 
                                             7:"U.S. Veterans Affairs", 
                                             8:"Other",
                                             9:"Other"
                                            }
                                           )

# convert formatting of States and change to abbreviations
mhs["State"] = mhs["State"].str.lower()
mhs["State"] = mhs["State"].str.title()

mhs["State"] = mhs["State"].replace({'Alabama': 'AL',
                                    'Alaska': 'AK',
                                    'Arizona': 'AZ',
                                    'Arkansas': 'AR',
                                    'California': 'CA',
                                    'Colorado': 'CO',
                                    'Connecticut': 'CT',
                                    'Delaware': 'DE',
                                    'Dist_Of_Columbia': 'DC',
                                    'Florida': 'FL',
                                    'Georgia': 'GA',
                                    'Hawaii': 'HI',
                                    'Idaho': 'ID',
                                    'Illinois': 'IL',
                                    'Indiana': 'IN',
                                    'Iowa': 'IA',
                                    'Kansas': 'KS',
                                    'Kentucky': 'KY',
                                    'Louisiana': 'LA',
                                    'Maine': 'ME',
                                    'Maryland': 'MD',
                                    'Massachusetts': 'MA',
                                    'Michigan': 'MI',
                                    'Minnesota': 'MN',
                                    'Mississippi': 'MS',
                                    'Missouri': 'MO',
                                    'Montana': 'MT',
                                    'Nebraska': 'NE',
                                    'Nevada': 'NV',
                                    'New_Hampshire': 'NH',
                                    'New_Jersey': 'NJ',
                                    'New_Mexico': 'NM',
                                    'New_York': 'NY',
                                    'North_Carolina': 'NC',
                                    'North_Dakota': 'ND',
                                    'Ohio': 'OH',
                                    'Oklahoma': 'OK',
                                    'Oregon': 'OR',
                                    'Pennsylvania': 'PA',
                                    'Rhode_Island': 'RI',
                                    'South_Carolina': 'SC',
                                    'South_Dakota': 'SD',
                                    'Tennessee': 'TN',
                                    'Texas': 'TX',
                                    'Utah': 'UT',
                                    'Vermont': 'VT',
                                    'Virginia': 'VA',
                                    'Washington': 'WA',
                                    'West_Virginia': 'WV',
                                    'Wisconsin': 'WI',
                                    'Wyoming': 'WY'})

# reorganize columns and remove STFIPS column
mhs = mhs[["CASEID", "Region", "State", "MHSuicide", "Ownership"]]
mhs.to_csv('Resources/mhs_subset.csv')

mhs.head()

,CASEID,Region,State,MHSuicide,Ownership
0,1,South,KY,No,Private non-profit
1,58,South,KY,Yes,State or Regional Government
2,118,South,KY,Yes,Private non-profit
3,191,South,KY,Yes,Private non-profit
4,198,South,KY,Yes,Other
